Hard Additional Requirement 2

The aim of this additional requirment is to analyse some other data sets.

The data set being analysed in this notebook is data that NASA has collected: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh. A discription of each of the columns is available at: https://www.kaggle.com/nasa/meteorite-landings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from ipywidgets import *
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D, get_test_data
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib import cm
from operator import itemgetter

import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

from ipywidgets import widgets 
from IPython.display import display
from plotly.graph_objs import *
from plotly.widgets import GraphWidget
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.offline.init_notebook_mode(connected=True)

import cufflinks as cf
init_notebook_mode(connected=True)

from scipy.misc import imread

First the data needs to be read in.

In [ ]:
df=pd.read_csv("Meteorite_Landings.csv")

Secondly the refining of the data is done, so any duplicate rows, or row which have a 'None' value is dropped.

In [ ]:
refinedNASA = df.copy()
refinedNASA = refinedNASA.dropna()
refinedNASA = refinedNASA.drop_duplicates()

Then the only two columns which have a limited number of possible values are checked, and any row which has invalid values also gets dropped. These possible values was specified in the links up above in the explanation of the NASA data.

In [ ]:
#possible valid values
fallarray = ['Fell', 'Found']
nametypearray = ['Valid', 'Relict']

#keeps track fo which rows are invalid so need to be removed
rowsToRemove = []

#for each column the values are all checked
rCounter = 0
falls = refinedNASA['fall']
for fa in falls:
    flag = 1
    for f in fallarray:
                if f == fa:
                    flag = 0
    if flag == 1:
        rowsToRemove.append(rCounter)
    rCounter = rCounter + 1

rCounter = 0
types = refinedNASA['nametype']  
for typ in types:
    flag = 1
    for name in nametypearray:
                if name == typ:
                    flag = 0
    if flag == 1:
        rowsToRemove.append(rCounter)
    rCounter = rCounter + 1
     
#list is reversed so that when the rows are getting removed it doesn't interfere with the indexes of other rows
rowsToRemove.sort(reverse=True)

for i in rowsToRemove:
    refinedNASA = refinedNASA.drop(refinedNASA.index[[i]])

#CSV file for the refined Data is made
refinedNASA.to_csv("refinedNASA.csv")

Finding all the possible values for each column and their occurances, all except 'name' and 'id'.

In [ ]:
nametype = refinedNASA.groupby('nametype')
nametype.size().sort_values()

In [ ]:
recclass = refinedNASA.groupby('recclass')
recclass.size().sort_values()

In [ ]:
fall = refinedNASA.groupby('fall')
fall.size().sort_values()

In [ ]:
year = refinedNASA.groupby('year')
year.size().sort_values()

In [ ]:
reclat = refinedNASA.groupby('reclat')
reclat.size().sort_values()

In [ ]:
reclong = refinedNASA.groupby('reclong')
reclong.size().sort_values()

In [ ]:
GeoLocation = refinedNASA.groupby('GeoLocation')
GeoLocation.size().sort_values()

This bar plot shows all the possible clas of metorite, 'recclass', and the number of records each one has. Double-click on the figure to see in detail.

In [ ]:
ax = refinedNASA['recclass'].value_counts().plot(kind="bar", figsize=(155, 150))
ax.set_xlabel("Recclass")
ax.set_ylabel("Count")
plt.show()

This line plot shows the masses, in grams, of each meteorite recorded.

In [ ]:
ax = refinedNASA[['mass (g)']].plot(kind="line", figsize=(10,10))
ax.set_xlabel("Record in Data-Frame")
ax.set_ylabel("Mass (g) 10s of Millions")
#setting the y axis to have more helpful axis values
ax.set_yticks([0, 5000000, 10000000,25000000,35000000, 50000000,60000000, 70000000])
plt.show()

In this next plot, a pie chart was used to show the number of metorites recorded in each year, the years which had less that 200 metorites were grouped together, to make the chart easier to see and to show all the labels clearly.

In [ ]:
colors = ['#009933', '#00cc66', '#009999', '#336699', '#003399', '#0099ff', '#00ffcc', '#3333cc', '#9999ff', '#99ccff']

data = refinedNASA.copy()
possibleYears = data.year.unique()
#keeping trach of all the rows to remove
rowsToRemove = []

years = data['year']  
for possible in possibleYears:
    rCounter = 0
    flag = 1
    tempRows = []
    for year in years:
                if possible == year:
                    flag = flag +1
                    if rCounter not in tempRows:
                        tempRows.append(rCounter)
                rCounter = rCounter + 1
    #if there were less than 200 occurances of this year then it gets added to the remove list
    if flag < 200:
        #makes sure the row doesn't get added twice to the remove list
        for num in tempRows:
            if num not in rowsToRemove:
                rowsToRemove.append(num)

#keeps track of how many lines to re-add once they've been dropped
linesToAdd = len(rowsToRemove)
rowsToRemove.sort(reverse=True)
for i in rowsToRemove:
    data = data.drop(data.index[[i]])
    
#adds the same number of rows as deleted all with the same value for 'year'
for i in range(0, linesToAdd):
    line = pd.DataFrame([[1,1,1,1,1,1, "Other Years" ,1,1,1]], columns=['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year', 'reclat', 'reclong', 'GeoLocation'])
    data = data.append(line, ignore_index=True)

data['year'].value_counts().plot(kind="pie", colors = colors, figsize=(10,10))
plt.show()

This interactive widget allows the user to look at the locations where Meteorites have fallen. To make it actually usable, the list of meteorites had to be limited, otherwise the list of them takes too long to generate and can sometimes crash the notebook due to the sheer number of records. The axes are as accurate as they could be made in accordance with the numbers on the map background image. The tab for the options available takes a bit of time to open, but if you give it a minute it will open.

In [ ]:
#the image used as the background
img = imread("world-latitudes.jpg")
name = list(refinedNASA.name.unique())
#limits the list to every 5th element, this can be changed to the full list, but it takes a while to collect all the data
names = name[::5]
#the function to make it interactive
def update(Meteorite = names):
    fig = plt.figure(figsize=(15,12))
    ax = fig.add_subplot(1, 1, 1)
    #gets the data for the Meteorite the user has selected
    data = refinedNASA[(((refinedNASA['name'] == Meteorite)))]
    #gets the 'GeoLocation' for the record
    geo = data.GeoLocation.iloc[0]
    numbers = geo.split("(")
    numbers = numbers[1].split(")")
    numbers = numbers[0].split(", ")
    #separates the two values fromt he data and stores them ready to be used
    latitude = numbers[0]
    longitude = numbers[1]
    #sets the values for the y axis to be as close to the onces on the background image
    ax.set_yticks([-80,-60, -40, 0, 40,60, 80])
    ax.imshow(img, extent=[(-175),175,(-90),90])
    #drawing the two lines for the lat value and long value
    ax.axhline(y=(float(latitude)), color='red')
    ax.axvline(x=(float(longitude)), color='red')
    fig.canvas.draw()

interact(update);